# Lesson 2: Array-oriented programming (workbook)

Import statements: run these first.

In [ ]:
# Python standard library
from functools import reduce
from itertools import combinations
import dis

# Scientific Python ecosystem
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from pympler.tracker import SummaryTracker

All of the mini-quizzes in this lesson come from [Array-oriented puzzles](https://github.com/hsf-training/array-oriented-puzzles), which has more challenging exercises as well, if you want to test your skills.

## Complex example of imperative, functional, and array-oriented

In [ ]:
def imperative_forces(m, x, p):
    total_force = np.zeros_like(x)

    for i in range(len(x)):
        for j in range(i + 1, len(x)):
            mi, mj = m[i], m[j]
            xi, xj = x[i], x[j]
            pi, pj = p[i], p[j]
            displacement = [
                xj[0] - xi[0],
                xj[1] - xi[1],
                xj[2] - xi[2],
            ]
            distance = np.sqrt(displacement[0]**2 + displacement[1]**2 + displacement[2]**2)
            direction = [
                displacement[0] / distance,
                displacement[1] / distance,
                displacement[2] / distance,
            ]
            force = [
                G * mi * mj * direction[0] / distance**2,
                G * mi * mj * direction[1] / distance**2,
                G * mi * mj * direction[2] / distance**2,
            ]
            total_force[i, 0] += force[0]
            total_force[i, 1] += force[1]
            total_force[i, 2] += force[2]
            total_force[j, 0] += -force[0]
            total_force[j, 1] += -force[1]
            total_force[j, 2] += -force[2]

    return total_force

In [ ]:
def functional_forces(m, x, p):
    def negate(vector):
        return [-a for a in vector]

    def add(*vectors):
        return [reduce(lambda a, b: a + b, components) for components in zip(*vectors)]

    def subtract(vectorA, vectorB):
        return add(vectorA, negate(vectorB))

    def magnitude(vector):
        return np.sqrt(reduce(lambda a, b: a + b, map(lambda a: a**2, vector)))

    def force(mi, mj, xi, xj, pi, pj):
        displacement = subtract(xi, xj)
        distance = magnitude(displacement)
        direction = [a / distance for a in displacement]
        return [G * mi * mj * a / distance**2 for a in direction]

    pairwise_forces = [
        ((i, j), force(mi, mj, xi, xj, pi, pj))
        for ((i, (mi, xi, pi)), (j, (mj, xj, pj))) in combinations(enumerate(zip(m, x, p)), 2)
    ]

    def partial_forces(pairwise_forces, i):
        return (
            [force for (_, check), force in pairwise_forces if i == check] +
            [negate(force) for (check, _), force in pairwise_forces if i == check]
        )

    return np.array([add(*partial_forces(pairwise_forces, i)) for i in range(len(m))])

In [ ]:
def array_forces(m, x, p):
    i, j = np.triu_indices(len(x), k=1)
    pw_displacement = x[j] - x[i]
    pw_distance = np.sqrt(np.sum(pw_displacement**2, axis=-1))
    pw_direction = pw_displacement / pw_distance[:, np.newaxis]
    pw_force = G * m[i, np.newaxis] * m[j, np.newaxis] * pw_direction / pw_distance[:, np.newaxis]**2
    total_force = np.zeros_like(x)
    np.add.at(total_force, i, pw_force)
    np.add.at(total_force, j, -pw_force)
    return total_force

In [ ]:
# three masses
m = np.array([2, 10, 1.1])

# initial position (x) and momentum (p) for each of the three
x = np.array([[0.1, 0.1, 0], [2, 0.9, 0.1], [-5, 1.5, -0.1]])
p = np.array([[3, -1, 0.5], [-13, 0, -0.2], [-10, 0.1, 0]])

G = 1

In [ ]:
imperative_forces(m, x, p)

In [ ]:
functional_forces(m, x, p)

In [ ]:
array_forces(m, x, p)

In [ ]:
m = np.ones(500)
x = np.random.normal(0, 1, (500, 3))
p = np.random.normal(0, 1, (500, 3))

In [ ]:
%%timeit -n1 -r1

imperative_forces(m, x, p)

In [ ]:
%%timeit -n1 -r1

functional_forces(m, x, p)

In [ ]:
%%timeit -n1 -r1

array_forces(m, x, p)

In [ ]:
def array_step(m, x, p, dt):
    # this is a numerically stable way of updating positions, momenta, and forces
    p += array_forces(m, x, p) * (dt/2)    # half kick
    x += p * dt / m[:, np.newaxis]         # full drift
    p += array_forces(m, x, p) * (dt/2)    # half kick

In [ ]:
def plot(m, x, p, dt, num_frames=100, steps_per_frame=10):
    num_particles = len(m)

    history = np.empty((num_frames, num_particles, 2))
    for i in range(num_frames):
        history[i, :, 0] = x[:, 0]
        history[i, :, 1] = x[:, 1]
        for _ in range(steps_per_frame):
            array_step(m, x, p, dt)

    fig, ax = plt.subplots(figsize=(5, 5))

    lines = []
    for j in range(num_particles):
        lines.append(ax.plot(history[:1, j, 0], history[:1, j, 1])[0])
    dots = ax.scatter(history[0, :, 0], history[0, :, 1])

    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)

    def update(i):
        for j, line in enumerate(lines):
            line.set_xdata(history[:i, j, 0])
            line.set_ydata(history[:i, j, 1])
        dots.set_offsets(history[i, :, :])
        return [*lines, dots]

    ani = animation.FuncAnimation(fig=fig, func=update, frames=num_frames, interval=50, blit=True)
    out = HTML(ani.to_jshtml())
    plt.close()
    return out

Double-planet orbits a star:

In [ ]:
m = np.array([100, 1, 1], np.float64)
x = np.array([[0, 0, 0], [0, 0.9, 0], [0, 1.1, 0]], np.float64)
p = np.array([[0, 0, 0], [-13, 0, 0], [-10, 0, 0]], np.float64)

plot(m, x, p, dt=0.001)

Solution to the three-body problem!

In [ ]:
a = 0.347111
b = 0.532728
m = np.array([1, 1, 1], np.float64)
x = np.array([[-1, 0, 0], [1, 0, 0], [0, 0, 0]], np.float64)
p = np.array([[a, b, 0], [a, b, 0], [-2 * a, -2 * b, 0]], np.float64)

plot(m, x, p, dt=0.01)

Chaos!

In [ ]:
m = np.ones(25)
x = np.random.normal(0, 1, (25, 3))
p = np.random.normal(0, 1, (25, 3))

plot(m, x, p, dt=0.0025)

## Example of an array-oriented conversation

In [ ]:
dataset = np.random.normal(0, 1, 1000000)  # one MILLION data points

In [ ]:
plt.hist(dataset, bins=100, range=(-5, 5));

In [ ]:
dataset2 = dataset**2

In [ ]:
plt.hist(dataset2, bins=100, range=(-5, 5));

In [ ]:
dataset3 = np.sin(1/dataset2)

In [ ]:
plt.hist(dataset3, bins=100, range=(-1, 1));

## NumPy features

<img src="img/Numpy_Python_Cheat_Sheet.svg" width="100%">

### Slicing

In [ ]:
arr = np.array([[1.1, 2.2, 3.3],
                [4.4, 5.5, 6.6],
                [7.7, 8.8, 9.9]])
arr

**Mini-quiz 1:** Given this 3D array,

In [ ]:
array3d = np.arange(2 * 3 * 5).reshape(2, 3, 5)
array3d

you can select items

<img src="img/array3d-highlight1.svg" width="25%">

with

In [ ]:
array3d[:, 1:, 1:]

Write a slice that selects these elements:

<img src="img/array3d-highlight2.svg" width="25%">

**Mini-quiz 2:** Use slicing and elementwise subtraction (`-`) together to find the sizes of the spaces between consecutive elements in the following array. (They're all `1.1`.)

In [ ]:
array = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
array

Hint:

<img src="img/flat-and-shifted-operations.svg" width="100%">

### Advanced slicing

In [ ]:
arr  = np.array([  0.0,   1.1,   2.2,   3.3,   4.4,  5.5,   6.6,  7.7,   8.8,  9.9])
mask = np.array([False, False, False, False, False, True, False, True, False, True])
#                                                    5.5          7.7          9.9

In [ ]:
arr[mask]

In [ ]:
text = """
WOULD YOU LIKE GREEN EGGS AND HAM?

I DO NOT LIKE THEM, SAM-I-AM.
I DO NOT LIKE GREEN EGGS AND HAM.

WOULD YOU LIKE THEM HERE OR THERE?

I WOULD NOT LIKE THEM HERE OR THERE.
I WOULD NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU LIKE THEM IN A HOUSE?
WOULD YOU LIKE THEN WITH A MOUSE?

I DO NOT LIKE THEM IN A HOUSE.
I DO NOT LIKE THEM WITH A MOUSE.
I DO NOT LIKE THEM HERE OR THERE.
I DO NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU EAT THEM IN A BOX?
WOULD YOU EAT THEM WITH A FOX?

NOT IN A BOX. NOT WITH A FOX.
NOT IN A HOUSE. NOT WITH A MOUSE.
I WOULD NOT EAT THEM HERE OR THERE.
I WOULD NOT EAT THEM ANYWHERE.
I WOULD NOT EAT GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.
"""

In [ ]:
# remove punctuation and make an array of words
words = np.array(text.replace(",", " ").replace(".", " ").replace("?", " ").replace("!", " ").replace("-", " ").split())

# find the unique words and get an index of where they are in the corpus
dictionary, index = np.unique(words, return_inverse=True)

### Reductions

In [ ]:
arr = np.array([[  1,   2,   3,   4],
                [ 10,  20,  30,  40],
                [100, 200, 300, 400]])

**Mini-quiz 3:** Using slicing, elementwise operations, and a reducer, find the length of this curve.

<img src="img/length-by-segment.svg" width="50%">

In [ ]:
t = np.linspace(0, 2*np.pi, 10000)
x = np.sin(3*t)
y = np.sin(4*t)